In [1]:
%load_ext autoreload
%autoreload 2
import os
import time
import cv2
import sys
sys.path.append("/home/yuval/Documents/XNOR/sealnet/models/darknet")
from pydn.inference.models import Detection
from pydn.inference.util import load, get_image_info_api, show_images, filter_confidence, save_dets, normalize_thermal
import pydn.register.models
from pydn.register.models import cent_h5_f, left_h5_f, RegistrationHelper, get_file_pairs
import importlib
import matplotlib.pyplot as plt
import numpy as np
# np.set_printoptions(suppress=True)
from IPython.display import display


In [29]:
class TransformMode:
    IRTOEO = 1
    EOTOIR = 2

class ModalityTransform():
    def __init__(self, H, mode):
        self.h = H
        self.mode = mode



    def transform_ir_to_eo(self, im_ir, im_eo):
        H = None
        if self.mode == TransformMode.IRTOEO:
            H = self.h
        elif self.mode == TransformMode.EOTOIR:
            H = np.linalg.inv(self.h)
        else:
            raise ("invalid mode")

        h, w, _ = im_eo.shape
        im_ir_3c = cv2.merge([im_ir, im_ir, im_ir])
        im_ir_3c[im_ir_3c == 0.] = 0.01
        im_proj = cv2.warpPerspective(im_ir_3c, H, (w, h))

        print(im_ir_3c.min())
        b_channel, g_channel, a_channel = cv2.split(im_proj)
        b_channel, g_channel, r_channel = cv2.split(im_eo)
        print(im_proj.min())

        # missing_mask = np.zeros(a_channel.shape)
        # missing_mask[a_channel == 0] = 1

        r_channel[a_channel == 0.] = 0.
        b_channel[a_channel == 0.] = 0.
        g_channel[a_channel == 0.] = 0.
        a_channel[a_channel == 0.] = 0.

        # r_channel[missing_mask_1c[:,1], missing_mask_1c[:,0]] = 255

        im_aligned = cv2.merge([b_channel, g_channel, r_channel, a_channel])
        return im_aligned  # , missing_mask

    def transform_eo_to_ir(self, im_eo, im_ir):
        H = None
        if self.mode == TransformMode.IRTOEO:
            H = np.linalg.inv(self.h)
        elif self.mode == TransformMode.EOTOIR:
            H = self.h
        else:
            raise ("invalid mode")

        h, w = im_ir.shape

        im_proj = cv2.warpPerspective(im_eo, H, (w, h))

        b_channel, g_channel, r_channel = cv2.split(im_proj)
        im_aligned = cv2.merge((b_channel, g_channel, r_channel, im_ir))
        return im_aligned


In [2]:
pkl_file_out_filter = '/fast/experiments/yolov4_3c_832/base/inference/kotz/kotz_fl01_fl04_fl05_detections_consolidated_nms0.20_conf0.10.pkl'

pkl_file_projected = '/fast/experiments/yolov4_3c_832/base/inference/kotz/projected/kotz_fl04_detections_consolidated_nms0.20_conf0.10_projected.pkl'
detections_dict = load(pkl_file_out_filter)
dirs = ['/data2/2019/fl04/CENT']
rh = RegistrationHelper(cent_h5_f)
rh_rev = RegistrationHelper(cent_h5_f, inverse=True)
print(rh.H)
print(rh_rev.H)
pairs = get_file_pairs(dirs[0])

[[ 1.06156124e+01 -4.68370028e-02 -2.58738196e+02]
 [ 6.50620827e-02  1.05816533e+01 -2.92569479e+02]]
[[ 1.06156124e+01 -4.68370028e-02 -2.58738196e+02]
 [ 6.50620827e-02  1.05816533e+01 -2.92569479e+02]]


/home/yuval/Documents/XNOR/sealnet/models/darknet/pydn/register/models.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  affine=np.array(data.value)


In [3]:
projections_dict_scaled = load(pkl_file_projected)

if projections_dict_scaled is None:
    projections_dict_scaled = {}

    total = len(detections_dict)
    for im in detections_dict:
        if not im in pairs:
            continue
        ir_im = pairs[im]
        if not im in projections_dict_scaled:
           projections_dict_scaled[im] = []
    #     if not ir_im in projections_dict_unscaled:
    #        projections_dict_unscaled[ir_im] = []

        rgb_name = os.path.basename(im)
        ir_name = os.path.basename(ir_im)
        rgb_info = get_image_info_api(rgb_name)
        ir_info = get_image_info_api(ir_name)
        scale_x =  rgb_info["width"] / ir_info["width"]
        scale_y = rgb_info["height"] / ir_info["height"]
        for det in detections_dict[im]:
            ((x1,y1),(x2,y2))=rh.project_box(det.box,scale_x,scale_y)
            p_det_scaled = Detection([x1,y1,x2,y2], det.classid, det.confidence)
            projections_dict_scaled[im].append(p_det_scaled)
    #         projections_dict_unscaled[ir_im].append(p_det_unscaled)
    save_dets(pkl_file_projected,projections_dict_scaled)

### Check to see if it worked!


In [65]:
from thebook.cv2.norm import min_max_norm
def show_projections(rgb_im, H):
    print(rgb_im)
    rgb = cv2.imread(rgb_im)
    height, width, channels = rgb.shape
    
    ir_im = pairs[rgb_im]
    ir=cv2.imread(ir_im, -1)
    ir = np.array(ir)
    ir = min_max_norm(ir)
    ir_3c = cv2.cvtColor(ir,cv2.COLOR_GRAY2RGB)
    print()
    #$rh.get_homography(cv2.MOTION_HOMOGRAPHY)
    ir_3c = cv2.warpPerspective(ir_3c, H, (width,height))
    print()
    # ir_3c = cv2.resize(ir_3c,None,fx=rgb.shape[1]/ir.shape[1],fy=rgb.shape[0]/ir.shape[0])

    for det in detections_dict[rgb_im]:
        [x1,y1,x2,y2] = det.box
        x_offset = -20
        y_offset = 80
        x_offset = -20
        y_offset = 60
        cv2.rectangle(ir_3c, (int(x1+x_offset), int(y1+y_offset)), (int(x2+x_offset),int(y2+y_offset)), (36, 255, 12), 8)
        cv2.rectangle(rgb, (int(x1), int(y1)), (int(x2),int(y2)), (36, 255, 12), 8)
    return ir_3c, rgb

def resize(img, scale_percent = 60):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [66]:
from noaadb import Session
from noaadb.schema.models import *
s = Session()
homography = s.query(Homography).filter(Homography.id == 64).first()
H = homography.matrix
print(H)
s.close()

[[ 1.06156124e+01 -4.68370028e-02 -2.58738196e+02]
 [ 6.50620827e-02  1.05816533e+01 -2.92569479e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [67]:
ims = list(projections_dict_scaled.keys())
count=0
for im in ims[27:28]:
    print(count)
    if count > 100: break
    ir, rgb = show_projections(im, H)
    count +=1
    cv2.imwrite('/home/yuval/Documents/XNOR/sealnet/models/darknet/pydn/register/test/%d_ir.jpg'%count,resize(ir,60),[int(cv2.IMWRITE_JPEG_QUALITY), 100])
    cv2.imwrite('/home/yuval/Documents/XNOR/sealnet/models/darknet/pydn/register/test/%d_rgb.jpg'%count,resize(rgb,40), [int(cv2.IMWRITE_JPEG_QUALITY), 100])

0
/data2/2019/fl04/CENT/test_kotz_2019_fl04_C_20190510_004343.407554_rgb.jpg




In [61]:
%matplotlib inline
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pylab as pl
from IPython.display import clear_output
from pylab import rcParams
rcParams['figure.figsize'] = 20,20

if projections_dict_scaled is None:
    projections_dict_scaled = load(pkl_file_projected)

ims = list(projections_dict_scaled.keys())
ims = [ims[5]]
fig2 = plt.figure(figsize = (10,10))
button1 = widgets.Button(description="Next")
button2 = widgets.Button(description="Prev")
out = widgets.Output()

buttons = widgets.VBox(children=[button1, button2])
all_widgets = widgets.HBox(children=[buttons, out])
display(all_widgets)
class ButtonCounter:
    def __init__(self, maxi):
        self.count = 0
        self.maxi = maxi
        
    def next(self,b):
        if self.count  < self.maxi:
            ir, rgb = show_projections(ims[self.count])
            self.count +=1
            print("test")
            cv2.imwrite('/home/yuval/Documents/XNOR/sealnet/models/darknet/pydn/register/test/%d_ir.jpg'%self.count,ir)
            cv2.imwrite('/home/yuval/Documents/XNOR/sealnet/models/darknet/pydn/register/test/%d_rgb.jpg'%self.count,rgb)
            with out:
                pl.imshow(ir)
                clear_output(wait=True)
                display(pl.gcf())
    def prev(self, b):
        if self.count  > 0:
            self.count -=1
            im = show_projections(ims[self.count])
            with out:
                pl.imshow(im)
                clear_output(wait=True)
                display(pl.gcf())
b = ButtonCounter(len(ims))
button1.on_click(b.next)
button2.on_click(b.prev)



<Figure size 720x720 with 0 Axes>

In [6]:
rgb = cv2.imread(im)
detections = detections_dict[im]
for det in detections:
    [x1,y1,x2,y2] = det.box
    cv2.rectangle(rgb, (int(x1), int(y1)), (int(x2),int(y2)), (255, 255, 0), 4)

plt.rcParams['figure.dpi'] = 200
plt.gca().set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())
plt.imshow(rgb)



NameError: name 'im' is not defined

array([0 0.9518 (x1:6409,y1:4240) (x2:6460,y2:4308),
       0 0.1653 (x1:242,y1:3066) (x2:259,y2:3118),
       0 0.1705 (x1:2761,y1:2272) (x2:2815,y2:2309),
       0 0.2332 (x1:2175,y1:1755) (x2:2199,y2:1806)], dtype=object)